In [20]:
# --- 1.0 INSTALLATIONS ---#
# Install libraries that don't come by default or need specific versions
!pip install gurobipy -q
# Note: The free version installed via pip allows models up to 2000 variables
# and 2000 constraints.

# --- 1.1 LIBRARIES ---
import pandas as pd
import requests
import re
import time
import os
import kagglehub
import pandas_gbq
import gurobipy as gp
from gurobipy import GRB
from google.cloud import bigquery
from google.colab import auth

In [21]:
# --- 1.2 CONFIGURATION & AUTHENTICATION ---#
# Authentication for Google Colab environment
auth.authenticate_user()

PROJECT_ID = 'YOUR-PROJECT-BIGQUERY'
API_TOKEN = 'YOUR-TOKEN' # Football-Data.org Token

# Initialize the BigQuery Client globally
client = bigquery.Client(project=PROJECT_ID)

print(f" Environment Ready. Project: {PROJECT_ID}")

 Environment Ready. Project: otimizador-cargas


In [22]:
#--- 2.0 HELPER FUNCTIONS ---#

# 2.1 ETL TO REMOVE DUPLICATES
def clean_and_deduplicate_columns(df):
    def clean_name(col):
        col = re.sub(r'[^\w]', '_', col).strip('_')
        return col if col else 'column'

    new_cols = [clean_name(c) for c in df.columns]
    final_cols = []
    counts = {}
    for col in new_cols:
        if col in counts:
            counts[col] += 1
            final_cols.append(f"{col}_{counts[col]}")
        else:
            counts[col] = 0
            final_cols.append(col)
    df.columns = final_cols
    return df


In [23]:
#--- 3.0 API FOOTBALL DATA COLLECTION (TOP 5 EUROPEAN LEAGUES) ---#
def fetch_api_data():
    # Free Tier Competition codes: PL (England), PD (Spain), BL1 (Germany), SA (Italy), FL1 (France)
    leagues = ['PL', 'PD', 'BL1', 'SA', 'FL1']
    headers = {'X-Auth-Token': API_TOKEN}
    all_players = []

    for league in leagues:
        # Corrected Endpoint for Teams within a Competition
        url = f"https://api.football-data.org/v4/competitions/{league}/teams"
        response = requests.get(url, headers=headers)

        if response.status_code == 200:
            data = response.json()
            teams = data.get('teams', [])
            league_name = data.get('competition', {}).get('name')

            for team in teams:
                squad = team.get('squad', [])
                for player in squad:
                    all_players.append({
                        'name': player.get('name'),
                        'position': player.get('position'),
                        'dateOfBirth': player.get('dateOfBirth'),
                        'nationality': player.get('nationality'),
                        'team_name': team.get('name'),
                        'league': league_name
                    })
            print(f"League {league} processed successfully.")
        elif response.status_code == 429:
            print(f"Rate limit hit for {league}. Waiting 60 seconds...")
            time.sleep(60)
            # Simple retry once
            response = requests.get(url, headers=headers)
            # (Process logic here if needed for retry)
        else:
            print(f"Error fetching {league}: {response.status_code} - {response.text}")

        # Free tier limit is often 10 calls per minute; wait a bit between leagues
        time.sleep(6)

    return pd.DataFrame(all_players)

# 3.1 EXECUTION AND UPLOAD
df_api = fetch_api_data()

if not df_api.empty:
    df_api_clean = clean_and_deduplicate_columns(df_api)

    # Destination Table
    TABLE_ID = 'sports.stg_api_players'

    pandas_gbq.to_gbq(
        df_api_clean,
        destination_table=TABLE_ID,
        project_id=PROJECT_ID,
        if_exists='replace'
    )
    print(f"Success: {len(df_api_clean)} players uploaded to {TABLE_ID}.")
else:
    print("No data collected. Check if you verified your email or if competition codes are available for your tier.")


League PL processed successfully.
League PD processed successfully.
League BL1 processed successfully.
League SA processed successfully.
League FL1 processed successfully.


100%|██████████| 1/1 [00:00<00:00, 12372.58it/s]

Success: 3118 players uploaded to sports.stg_api_players.


In [24]:
# 3.2 COLLECTING TRANSFERMARKET AND PERFORMANCE DATA (KAGGLE)

# --- Transfermarkt  ---
try:
    print("Transfermarkt...")
    path_tm = kagglehub.dataset_download("davidcariboo/player-scores")#RAW DATA ON KAGGLE
    df_players = pd.read_csv(os.path.join(path_tm, "players.csv"))
    pandas_gbq.to_gbq(clean_and_deduplicate_columns(df_players), 'dadostransfermarkt.players', PROJECT_ID, if_exists='replace')#CLEAN AND UPLOAD DATA TO BIGQUERY
    print(" Transfermarkt OK.")
except Exception as e:
    print(f" Erro Transfermarkt: {e}")

# --- Performance 24-25 ---
try:
    print("Performance 24-25...")
    path_2425 = kagglehub.dataset_download("hubertsidorowicz/football-players-stats-2024-2025")#RAW DATA ON KAGGLE
    df_2425 = pd.read_csv(os.path.join(path_2425, "players_data-2024_2025.csv"))
    pandas_gbq.to_gbq(clean_and_deduplicate_columns(df_2425), 'sports.stg_tech_stats_2425', PROJECT_ID, if_exists='replace')#CLEAN AND UPLOAD DATA TO BIGQUERY
    print(" 24-25 OK.")
except Exception as e:
    print(f" Erro 24-25: {e}")

print("\n Verifique seu BigQuery!")

Transfermarkt...
Using Colab cache for faster access to the 'player-scores' dataset.


100%|██████████| 1/1 [00:00<00:00, 4169.29it/s]


 Transfermarkt OK.
Performance 24-25...
Using Colab cache for faster access to the 'football-players-stats-2024-2025' dataset.


100%|██████████| 1/1 [00:00<00:00, 12336.19it/s]

 24-25 OK.

 Verifique seu BigQuery!


In [26]:
#--- 4.0 DATA PREPARATION ---#

# We connect all tables by player name and aggregate to make sure there is only 1 line by name, could be done with windows functions too
sql_query = f"""
CREATE OR REPLACE TABLE `{PROJECT_ID}.sports.optimization_final_input` AS
WITH api_positions AS (
    SELECT DISTINCT
        LOWER(name) as join_name,
        ANY_VALUE(name) as player_name,
        ANY_VALUE(position) as api_pos,
        ANY_VALUE(team_name) as team_name,
        ANY_VALUE(league) as league,
        ANY_VALUE(nationality) as nationality
    FROM `{PROJECT_ID}.sports.stg_api_players`
    GROUP BY 1
),
tech_stats AS (
    SELECT DISTINCT
        LOWER(Player) as join_name,
        -- Renaming technical stats to descriptive names
        SUM(Gls) as goals,
        SUM(Ast) as assists,
        SUM(TklW) as tackles_won,
        SUM(Int) as interceptions,
        SUM(Blocks) as blocks,
        SUM(Clr) as clearances,
        SUM(CrsPA) as crosses_into_penalty_area,
        SUM(PrgP) as progressive_passes,
        SUM(PrgC) as progressive_carries,
        SUM(SCA) as shot_creation_actions,
        SUM(KP) as key_passes,
        SUM(Succ) as dribbles_completed,
        SUM(SoT) as shots_on_target,
        SUM(xG) as expected_goals,
        SUM(xAG) as expected_assisted_goals,
        SUM(COALESCE(Saves, 0)) as goalkeeper_saves,
        SUM(Min) as minutes_played
    FROM `{PROJECT_ID}.sports.stg_tech_stats_2425`
    GROUP BY 1
),
market_values AS (
    SELECT DISTINCT
        LOWER(name) as join_name,
        SUM(market_value_in_eur / 1000000) as market_value_mio
    FROM `{PROJECT_ID}.dadostransfermarkt.players`
    GROUP BY 1
)
---## 4.1 PERFOMANCE CALCULATIONS
---# On this section we prepare the weight distribuition who will be used by the optimization to find the best team in the budget, each stats may differ on weight depending on player position
SELECT
    DISTINCT api.player_name,
    api.api_pos,
    api.team_name,
    api.league,
    api.nationality,
    m.market_value_mio,
    t.* EXCEPT(join_name),
    -- CUSTOM PERFORMANCE SCORING LOGIC (Using renamed columns) 20 as weight distribuition
    CASE
        WHEN api.api_pos = 'Goalkeeper' THEN (t.goalkeeper_saves * 20)

        WHEN api.api_pos IN ('Right-Back', 'Left-Back') THEN
            (t.tackles_won * 8) + (t.crosses_into_penalty_area * 6) + (t.progressive_carries * 6)

        WHEN api.api_pos = 'Centre-Back' THEN
            (t.tackles_won * 8) + (t.interceptions * 4) + (t.blocks * 4) + (t.clearances * 4)

        WHEN api.api_pos = 'Defensive Midfield' THEN
            (t.tackles_won * 6) + (t.interceptions * 6) + (t.progressive_passes * 4) + (t.key_passes * 4)

        WHEN api.api_pos = 'Central Midfield' THEN
            (t.key_passes * 4) + (t.interceptions * 4) + (t.progressive_passes * 6) + (t.shot_creation_actions * 3) + (t.assists * 3)

        WHEN api.api_pos = 'Attacking Midfield' THEN
            (t.key_passes * 4) + (t.assists * 6) + (t.progressive_passes * 6) + (t.shot_creation_actions * 4)

        WHEN api.api_pos IN ('Right Winger', 'Left Winger', 'Offence') THEN
            (t.dribbles_completed * 4) + (t.crosses_into_penalty_area * 3) + (t.expected_assisted_goals * 3) + (t.goals * 5) + (t.assists * 5)

        WHEN api.api_pos = 'Centre-Forward' THEN
            (t.goals * 8) + (t.shots_on_target * 4) + (t.expected_goals * 4) + (t.assists * 4)

        ELSE (t.goals * 4 + t.assists * 4 + t.tackles_won * 2 + t.interceptions * 2 + t.progressive_passes * 2 + t.shot_creation_actions * 2 + t.dribbles_completed * 2 + t.key_passes * 2)
    END as performance_score
FROM api_positions api
INNER JOIN tech_stats t ON api.join_name = t.join_name
INNER JOIN market_values m ON api.join_name = m.join_name
WHERE t.minutes_played > 270
AND m.market_value_mio > 0;
"""

print("Building consolidated dataset with descriptive columns...")
client.query(sql_query).result()
print("Success: 'optimization_final_input' table created.")

Building consolidated dataset with descriptive columns...
Success: 'optimization_final_input' table created.


In [27]:
#--- 5.0 CHECKING THE FINAL BASE ---#
df_check = client.query(f"SELECT * FROM `{PROJECT_ID}.sports.optimization_final_input` ORDER BY performance_score DESC LIMIT 20").to_dataframe()
display(df_check)

,player_name,api_pos,team_name,league,nationality,market_value_mio,goals,assists,tackles_won,interceptions,...,progressive_carries,shot_creation_actions,key_passes,dribbles_completed,shots_on_target,expected_goals,expected_assisted_goals,goalkeeper_saves,minutes_played,performance_score
0,Bruno Fernandes,Attacking Midfield,Manchester United FC,Premier League,Portugal,40.0,8,10,50,26,...,77,193,91,28,26,9.9,8.5,0.0,3018,3146.0
1,Pedri,Central Midfield,FC Barcelona,Primera Division,Spain,80.0,4,5,34,26,...,92,165,70,40,11,2.2,7.4,0.0,2879,3054.0
2,Mark Flekken,Goalkeeper,Bayer 04 Leverkusen,Bundesliga,Netherlands,12.0,0,2,4,0,...,0,10,3,1,0,0.0,0.5,150.0,3275,3000.0
3,Joan García,Goalkeeper,FC Barcelona,Primera Division,Spain,30.0,0,0,0,0,...,0,9,0,0,1,0.1,0.0,140.0,3420,2800.0
4,Yehvann Diouf,Goalkeeper,OGC Nice,Ligue 1,France,12.0,0,0,1,1,...,0,7,0,2,1,0.1,0.0,131.0,3060,2620.0
5,Sergio Herrera,Goalkeeper,CA Osasuna,Primera Division,Spain,3.0,0,0,1,0,...,0,6,0,1,0,0.0,0.0,131.0,3330,2620.0
6,Vanja Milinković-Savić,Goalkeeper,SSC Napoli,Serie A,Serbia,18.0,0,0,0,0,...,0,10,3,0,0,0.0,0.1,129.0,3330,2580.0
7,Romano Schmid,Attacking Midfield,SV Werder Bremen,Bundesliga,Austria,17.0,5,4,25,11,...,107,151,68,45,13,4.3,6.2,0.0,2834,2574.0
8,Wladimiro Falcone,Goalkeeper,US Lecce,Serie A,Italy,5.0,0,0,0,0,...,0,11,0,0,0,0.0,0.0,125.0,3420,2500.0
9,Cole Palmer,Attacking Midfield,Chelsea FC,Premier League,England,120.0,15,8,20,11,...,120,202,87,51,44,17.3,10.9,0.0,3191,2488.0


In [28]:
df_check = client.query(f"SELECT COUNT(player_name), api_pos FROM `{PROJECT_ID}.sports.optimization_final_input` GROUP BY api_pos").to_dataframe()
display(df_check)

,f0_,api_pos
0,69,Attacking Midfield
1,175,Central Midfield
2,232,Centre-Back
3,135,Centre-Forward
4,13,Defence
5,90,Defensive Midfield
6,107,Goalkeeper
7,8,Left Midfield
8,74,Left Winger
9,89,Left-Back


In [29]:
#--- 6.0 OPTIMIZATION ---#

query = f"SELECT * FROM `{PROJECT_ID}.sports.optimization_final_input`"
df = client.query(query).to_dataframe()

# 6.1 INITIALIZE MODEL
m = gp.Model("Football_Optimization")

# 6.2 Decision Variables: x[i] is 1 if player i is selected, 0 otherwise
players = df.index
x = m.addVars(players, vtype=GRB.BINARY, name="x")

# 6.3 Objective Function: Maximize total performance_score
m.setObjective(gp.quicksum(df.loc[i, 'performance_score'] * x[i] for i in players), GRB.MAXIMIZE)

# 6.4. Constraints
# Constraint 1: Budget (Total Market Value <= 200 Million)
m.addConstr(gp.quicksum(df.loc[i, 'market_value_mio'] * x[i] for i in players) <= 200, name="Budget")

# Constraint 2: Total Players = 11
m.addConstr(gp.quicksum(x[i] for i in players) == 11, name="TotalPlayers")

# Constraint 3: Specific Position Counts (Your 1-4-3-3 Target)
m.addConstr(gp.quicksum(x[i] for i in players if df.loc[i, 'api_pos'] == 'Goalkeeper') == 1, name="GK")
m.addConstr(gp.quicksum(x[i] for i in players if df.loc[i, 'api_pos'] == 'Right-Back') == 1, name="RB")
m.addConstr(gp.quicksum(x[i] for i in players if df.loc[i, 'api_pos'] == 'Left-Back') == 1, name="LB")
m.addConstr(gp.quicksum(x[i] for i in players if df.loc[i, 'api_pos'] == 'Centre-Back') == 2, name="CB")
m.addConstr(gp.quicksum(x[i] for i in players if df.loc[i, 'api_pos'] == 'Defensive Midfield') == 1, name="DM")
m.addConstr(gp.quicksum(x[i] for i in players if df.loc[i, 'api_pos'] == 'Central Midfield') == 1, name="CM")
m.addConstr(gp.quicksum(x[i] for i in players if df.loc[i, 'api_pos'] == 'Attacking Midfield') == 1, name="AM")
m.addConstr(gp.quicksum(x[i] for i in players if df.loc[i, 'api_pos'] == 'Right Winger') == 1, name="RW")
m.addConstr(gp.quicksum(x[i] for i in players if df.loc[i, 'api_pos'] == 'Left Winger') == 1, name="LW")
m.addConstr(gp.quicksum(x[i] for i in players if df.loc[i, 'api_pos'] == 'Centre-Forward') == 1, name="CF")

# 6.5. Run Optimization
m.optimize()


Gurobi Optimizer version 13.0.1 build v13.0.1rc0 (linux64 - "Ubuntu 22.04.5 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 12 rows, 1250 columns and 3673 nonzeros (Max)
Model fingerprint: 0xf6b17faf
Model has 1250 linear objective coefficients
Variable types: 0 continuous, 1250 integer (1250 binary)
Coefficient statistics:
  Matrix range     [1e-01, 2e+02]
  Objective range  [4e-01, 3e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+02]

Found heuristic solution: objective 6036.8000000
Presolve removed 0 rows and 1066 columns
Presolve time: 0.01s
Presolved: 12 rows, 184 columns, 365 nonzeros
Found heuristic solution: objective 9340.1000000
Variable types: 0 continuous, 184 integer (183 binary)
Found heuristic solution: objective 11058.600000

Root relaxation: objective 1.736580e+04, 19 iterations, 0.00 seconds (0.00 work units)

    N

In [30]:
#--- 7.0 DISPLAY RESULTS ---#
if m.status == GRB.OPTIMAL:
    selected_indices = [i for i in players if x[i].x > 0.5]
    dream_team = df.loc[selected_indices]

    print("\n--- YOUR OPTIMIZED DREAM TEAM ---")
    print(dream_team[['player_name', 'api_pos', 'team_name', 'market_value_mio', 'performance_score']])
    print(f"\nTotal Cost: €{dream_team['market_value_mio'].sum():.2f} Million")
    print(f"Total Performance Rating: {dream_team['performance_score'].sum():.2f}")
else:
    print("No optimal solution found. Consider increasing the budget or adjusting constraints.")


--- YOUR OPTIMIZED DREAM TEAM ---
             player_name             api_pos              team_name  \
67       Bruno Fernandes  Attacking Midfield   Manchester United FC   
243                Pedri    Central Midfield           FC Barcelona   
295       Ardian Ismajli         Centre-Back              Torino FC   
432       Patrick Mainka         Centre-Back  1. FC Heidenheim 1846   
564   Robert Lewandowski      Centre-Forward           FC Barcelona   
712         Granit Xhaka  Defensive Midfield         Sunderland AFC   
813         Mark Flekken          Goalkeeper    Bayer 04 Leverkusen   
878           Luis Rioja         Left Winger            Valencia CF   
980     Antonee Robinson           Left-Back              Fulham FC   
1122               Sávio        Right Winger     Manchester City FC   
1233         Jesús Areso          Right-Back          Athletic Club   

      market_value_mio  performance_score  
67                40.0             3146.0  
243               80.0  